In [23]:
import arcpy, os, sys
import pandas as pd
arcpy.env.overwriteOutput = True

gdb = r'C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'

target_table = r'C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Test_Table'

datasets = []

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)
print(datasets)

datset_count = len(datasets)
dataset_paths = []

def dataset_path(x,y):
    path = os.path.join(x,y)
    return path

for ds in datasets:
    ds_path = dataset_path(gdb,ds)
    dataset_paths.append(ds_path)

print(dataset_paths)

field_list = []
table_fields = arcpy.ListFields(target_table)
for fld in table_fields:
    field_list.append(fld.name)
    print(fld.type)
print(field_list)

['Point', 'Polygon', 'Polyine']
['C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Point', 'C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Polygon', 'C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Polyine']
OID
String
String
Double
String
Date
String
String
String
String
String
String
String
String
String
String
String
String
Double
Double
String
String
Integer
String
String
String
Double
String
String
String
String
String
String
String
String
String
Double
String
String
String
String
String
String
String
String
String
String
String
String
String
String
Double
String
Double
Double
String
Date
String
String
String
Double
Double
Double
String
String
Date
String
Double
String
Double
Double
String
['OBJECTID', 'test_fld', 'ACQ_TYPE', 'ACRES', 'Address', 'CLOSE_DATE', 'COMMENTS', 'COMMENTS1', 'COMMENT

In [24]:
new_fields = []
field_dictionary = {}
fc_all = []
type_list = ["String","Single","Double","SmallInteger","Integer","Date","Blob","Raster","Guid"]
existing_fields_lower = []

for name in field_list:
    existing_fields_lower.append(name.lower())
print(existing_fields_lower)

for dsp in dataset_paths:
    env(dsp)
    print(dsp)
    fclist = arcpy.ListFeatureClasses()

    for fc in fclist:
        fc_desc = arcpy.Describe(fc)
        fc_path = os.path.join(dsp,fc_desc.name)
        fc_all.append(fc_path)
print(fc_all)

['objectid', 'test_fld', 'acq_type', 'acres', 'address', 'close_date', 'comments', 'comments1', 'comments2', 'coowners', 'county', 'city', 'desc1', 'desc2', 'digbnd_src', 'dig_com', 'esmt_hold', 'facility_t', 'ff_ac_pur', 'ff_funds_s', 'ff_pur_pr1', 'ff_pur_pr2', 'id', 'inferred', 'in_flma', 'longlat', 'ltf_acres', 'macode_bcd', 'majorma', 'manager', 'managing_a', 'maname', 'maname_ab', 'man_agncy', 'matype', 'matype2', 'ma_id', 'ma_name', 'ma_website', 'mgrcity', 'mgrinst', 'mgrphone', 'naics_code', 'naics_desc', 'name', 'oth_pur_pr', 'owner', 'ownership', 'ownertypes', 'parcelid', 'par_name', 'prc_ff', 'prj_name', 'protstat', 'shape__len', 'source', 'sourcedate', 'status', 'sub_1', 'sub_2', 'shape_area', 'shape_length', 'totacres', 'type', 'usng', 'val_date', 'val_method', 'voltage', 'volt_class', 'x', 'y', 'zip']
C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Point
C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_

In [25]:
arcpy.env.overwriteOutput = True
test_sheet = r'C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\Test_table.xlsx'
excel_writer = pd.ExcelWriter(test_sheet, engine='xlsxwriter')

def word_split(x):
    fc_words = x.split("_")
    fc_word_letters = [word[0] for word in fc_words]
    fc_name_abbr = ''.join(fc_word_letters)
    return fc_name_abbr

for fc in fc_all:
    fc_desc = arcpy.Describe(fc)
    fc_name = fc_desc.name

    table = arcpy.TableToTable_conversion(fc, 'in_memory', word_split(fc_name))
    df = pd.DataFrame([row for row in arcpy.da.SearchCursor(table, "*")])
    df.columns = [field.name for field in arcpy.ListFields(table)]
    df.to_excel(excel_writer, sheet_name=word_split(fc_name), index=False)

# Save the Excel file
excel_writer.save()

In [51]:
import pandas as pd

combined_df = pd.DataFrame()
test_sheet = r'C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\Test_table_1.xlsx'
excel_writer = pd.ExcelWriter(test_sheet, engine='xlsxwriter')

# Iterate through the feature classes
for fc in fc_all:    
    # Get the attribute fields
    field_names = [field.name for field in arcpy.ListFields(fc_path)]
    
    # Convert feature class data to a Pandas DataFrame
    data = [row for row in arcpy.da.SearchCursor(fc_path, field_names)]
    fc_df = pd.DataFrame(data, columns=field_names)
    
    # Append the feature class DataFrame to the combined DataFrame
    combined_df = pd.concat([combined_df, fc_df], axis=0)

# Group columns by name and count occurrences
column_counts = combined_df.columns.value_counts()

# Filter columns with more than one instance
selected_columns = column_counts[column_counts > 0].index.tolist()
print(selected_columns)
selected_df = combined_df[selected_columns]

selected_df.to_excel(excel_writer, index=False)

['OBJECTID_1', 'Shape', 'SHAPE__Len', 'SUB_2', 'SUB_1', 'INFERRED', 'VOLT_CLASS', 'VOLTAGE', 'OWNER', 'VAL_DATE', 'VAL_METHOD', 'SOURCEDATE', 'SOURCE', 'NAICS_DESC', 'NAICS_CODE', 'STATUS', 'TYPE', 'ID', 'OBJECTID', 'Shape_Length']


In [23]:
new_fields = []
field_dictionary = {}
fc_all = []
type_list = ["String","Single","Double","SmallInteger","Integer","Date","Blob","Raster","Guid"]
existing_fields_lower = []

for name in field_list:
    existing_fields_lower.append(name.lower())
print(existing_fields_lower)

for dsp in dataset_paths:
    env(dsp)
    print(dsp)
    fclist = arcpy.ListFeatureClasses()

    for fc in fclist:
        fc_desc = arcpy.Describe(fc)
        fc_path = os.path.join(dsp,fc_desc.name)
        fc_all.append(fc_path)
        fields = arcpy.ListFields(fc)

        for fld in fields:
            fld_name = fld.name
            fld_name_lower = fld_name.lower()

            if fld_name not in field_list:
                if fld_name not in new_fields:
                    if fld_name_lower not in existing_fields_lower:
                        if fld.type in type_list:
                            if (fld.name, fld.type) not in field_dictionary.items():
                                fld_name = fld.name
                                fld_name_lower = fld_name.lower()
                                field_dictionary[fld_name] = fld.type
                                new_fields.append(fld_name)
                                existing_fields_lower.append(fld_name_lower)


existing_fields_lower_sort = sorted(existing_fields_lower)
new_fld_sort = sorted(new_fields)
print(fc_all)
print(existing_fields_lower_sort)
print(len(existing_fields_lower_sort))
print(new_fld_sort)
print(len(new_fld_sort))





        

['objectid', 'test_fld']
C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Point
C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Polygon
C:\Users\matthew.cole\Documents\Project\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Polyine
['C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Point\\Projected_community_centers', 'C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Polygon\\Projected_conservation_lands', 'C:\\Users\\matthew.cole\\Documents\\Project\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Polyine\\Projected_electric_production_supply_facilities_transmission']
['acq_type', 'acres', 'address', 'city', 'close_date', 'comments', 'comments1', 'comments2', 'coowners', 'county', 'desc1', 'desc2', 'dig_com', 'digbnd_src', 'es

In [24]:
for flds in new_fld_sort:
    arcpy.AddField_management(target_table,flds,(field_dictionary[flds]))

new_tbl = arcpy.ListFields(target_table)
new_tbl_flds = []
for tbl_fld in new_tbl:
    new_tbl_flds.append(tbl_fld.name)
print(new_tbl_flds)
print(len(new_tbl_flds))

['OBJECTID', 'test_fld', 'ACQ_TYPE', 'ACRES', 'Address', 'CLOSE_DATE', 'COMMENTS', 'COMMENTS1', 'COMMENTS2', 'COOWNERS', 'COUNTY', 'City', 'DESC1', 'DESC2', 'DIGBND_SRC', 'DIG_COM', 'ESMT_HOLD', 'FACILITY_T', 'FF_AC_PUR', 'FF_FUNDS_S', 'FF_PUR_PR1', 'FF_PUR_PR2', 'ID', 'INFERRED', 'IN_FLMA', 'LONGLAT', 'LTF_ACRES', 'MACODE_BCD', 'MAJORMA', 'MANAGER', 'MANAGING_A', 'MANAME', 'MANAME_AB', 'MAN_AGNCY', 'MATYPE', 'MATYPE2', 'MA_ID', 'MA_NAME', 'MA_WEBSITE', 'MGRCITY', 'MGRINST', 'MGRPHONE', 'NAICS_CODE', 'NAICS_DESC', 'Name', 'OTH_PUR_PR', 'OWNER', 'OWNERSHIP', 'OWNERTYPES', 'PARCELID', 'PAR_NAME', 'PRC_FF', 'PRJ_NAME', 'PROTSTAT', 'SHAPE__Len', 'SOURCE', 'SOURCEDATE', 'STATUS', 'SUB_1', 'SUB_2', 'Shape_Area', 'Shape_Length', 'TOTACRES', 'TYPE', 'USNG', 'VAL_DATE', 'VAL_METHOD', 'VOLTAGE', 'VOLT_CLASS', 'X', 'Y', 'Zip']


In [44]:
# Create an insert cursor for the output table
in_fld_final = []
qc_fld_list = []
for fcs in fc_all:
    in_flds = arcpy.ListFields(fcs)
    for in_fld in in_flds:
        in_fld_name = in_fld.name
        if (in_fld_name, in_fld.type) in field_dictionary.items():
            in_fld_final.append(in_fld_name)
            qc_fld_list.append(in_fld_name)
print(qc_fld_list)
print(len(qc_fld_list))

with arcpy.da.InsertCursor(target_table, in_fld_final) as insert_cursor:
    # Loop through each feature class and insert its rows into the output table
    with arcpy.da.SearchCursor(fcs, in_fld_final) as search_cursor:
        for row in search_cursor:
            insert_cursor.insertRow(row)


['Name', 'Address', 'City', 'Zip', 'COUNTY', 'X', 'Y', 'LONGLAT', 'USNG', 'OWNERSHIP', 'PARCELID', 'FACILITY_T', 'MA_ID', 'MACODE_BCD', 'MANAME', 'MAJORMA', 'MATYPE', 'MANAGING_A', 'MATYPE2', 'OWNER', 'COOWNERS', 'OWNERTYPES', 'TOTACRES', 'LTF_ACRES', 'ESMT_HOLD', 'COUNTY', 'PROTSTAT', 'DESC1', 'DESC2', 'COMMENTS1', 'COMMENTS2', 'MANAGER', 'MGRINST', 'MGRCITY', 'MGRPHONE', 'MA_WEBSITE', 'MANAME_AB', 'DIGBND_SRC', 'ID', 'PAR_NAME', 'PRJ_NAME', 'MA_NAME', 'FF_PUR_PR1', 'FF_PUR_PR2', 'OTH_PUR_PR', 'FF_AC_PUR', 'ACQ_TYPE', 'CLOSE_DATE', 'MAN_AGNCY', 'COMMENTS', 'DIG_COM', 'IN_FLMA', 'FF_FUNDS_S', 'PRC_FF', 'ACRES', 'Shape_Length', 'Shape_Area', 'TYPE', 'STATUS', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'OWNER', 'VOLTAGE', 'VOLT_CLASS', 'INFERRED', 'SUB_1', 'SUB_2', 'SHAPE__Len', 'Shape_Length']
73


RuntimeError: Cannot find field 'Name'

In [ ]:
# Create an insert cursor for the output table
in_fld_final = []
qc_fld_list = []
with arcpy.da.InsertCursor(target_table, new_tbl_flds) as insert_cursor:
    # Loop through each feature class and insert its rows into the output table
    for fcs in fc_all:
        in_flds = arcpy.ListFields(fcs)
        for in_fld in in_flds:
            in_fld_name = in_fld.name
            in_fld_name_lower = in_fld_name.lower()
            if fld_name in new_tbl_flds:
                if in_fld_name_lower in existing_fields_lower:
                    if in_fld.type in type_list:
                        if (in_fld_name, in_fld.type) in field_dictionary.items():
                            qc_fld_list.extend(in_fld_final)
                            print(qc_fld_list)
                            with arcpy.da.SearchCursor(fcs, in_fld_final) as search_cursor:
                                for row in search_cursor:
                                    insert_cursor.insertRow(row)
print(qc_fld_list)
print(len(qc_fld_list))

In [49]:
pip install XlsxWriter

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 153.0/153.0 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
